In [1]:
import sys
sys.path.append("/home/ubuntu/caffe/python")
import caffe
import leveldb
import numpy as np
from caffe.proto import caffe_pb2

import pandas as pd
import sklearn
from sklearn import cross_validation
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC, LinearSVC
import matplotlib.pyplot as plt
import pickle

from numpy import genfromtxt

db = leveldb.LevelDB('./features_train_resnet-152/')

datum = caffe_pb2.Datum()
input_x = np.zeros((0,2048))

i=0
for key, value in db.RangeIter():
    datum.ParseFromString(value)
    data = caffe.io.datum_to_array(datum)
    input_x = np.vstack((input_x,data.reshape(-1,1).T))
    #print("training"+key)
    i+=1
    if i%1000 == 0:
        print("training"+key)
print(input_x.shape)


db = leveldb.LevelDB('./features_test_resnet-152/')
datum = caffe_pb2.Datum()
test_x = np.zeros((0,2048))

for key, value in db.RangeIter():
    datum.ParseFromString(value)
    data = caffe.io.datum_to_array(datum)
    test_x = np.vstack((test_x,data.reshape(-1,1).T))
    #print("test"+key)
print(test_x.shape)


input_label = genfromtxt('./Files/train.csv', delimiter=',')
input_label = input_label[1:7001,1].reshape(-1)

print ('input_x shape ',input_x.shape)
print ('input_label shape ',input_label.shape)

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


training0000000999
training0000001999
training0000002999
training0000003999
training0000004999
training0000005999
training0000006999
(7000, 2048)
(970, 2048)
('input_x shape ', (7000, 2048))
('input_label shape ', (7000,))


In [4]:
#add cross validation
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(input_x, input_label, test_size=0.1, random_state=42)
############
#print(X_train.shape)
###########

#change clf to multi class logistic regression
#clf = LogisticRegression(verbose=1,max_iter=100)

#change clf multi class method to crammer singer
#clf = LinearSVC(C=100.0, loss='hinge', penalty='l2',multi_class='ovr')

#change to use SVC instead of linearSVC
clf = SVC(C=500.0,decision_function_shape='ovr',max_iter=1000)

# SVC
# clf = SVC(C=100, cache_size=500, coef0=0.0, degree=3, gamma=1.0000000000000001e-04,
#          kernel='rbf',max_iter=-1, probability=False, random_state=None, shrinking=True, tol=0.001, verbose=False)

#add training accuracy for when training on entire input
#clf.fit(input_x, input_label)
#print('training accuracy is', clf.score(input_x,input_label))

#try neural network
#clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 10), random_state=1)

#fit of cross validation
print('fitting linear svm to training')
#for i in range(50):
clf.fit(X_train, y_train)
#    print('training accuracy is', clf.score(X_train,y_train))
#add training accuracy for when training on x_train
print('training accuracy is', clf.score(X_train,y_train))
#add validation accuracy for when training on x_train
print('validation accuracy is', clf.score(X_test,y_test))

# clf = LinearSVC(C=1.0, loss='squared_hinge', penalty='l2',multi_class='ovr')
# clf.fit(input_x, input_label)

#fake validation accuracy and training accuracy
# print('validation accuracy is', clf.score(X_test,y_test))
# print('training accuracy is', clf.score(X_train,y_train))

# y_pred = clf.predict(test_x)

# filename = "predict_resnet.csv"
# f = open(filename, "w")
# f.write('Id,Prediction\n')

# if ((len(y_pred))<1000):
#     zeros = np.zeros(2000)
#     y_pred = np.append(y_pred, zeros).reshape(-1)

# for i in range(0,len(y_pred)):
#     d = '{0},{1}\n'.format(i+1,int(y_pred[i]))
#     f.write(d)


fitting linear svm to training
('training accuracy is', 1.0)
('validation accuracy is', 0.81714285714285717)
